In [78]:
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
from os import listdir
from os.path import isfile, join
import os
from matplotlib.image import imread
import matplotlib.pyplot as plt
import scipy.io.wavfile as wav
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv2D, Flatten, MaxPooling2D
import librosa

In [2]:
dir_path = '/home/duh17/Desktop/free-spoken-digit-dataset/recordings/'
file_names = [f for f in listdir(dir_path) if isfile(join(dir_path, f)) and '.wav' in f]

['8_yweweler_24.wav',
 '4_jackson_36.wav',
 '0_yweweler_36.wav',
 '8_nicolas_1.wav',
 '5_nicolas_22.wav',
 '0_jackson_5.wav',
 '6_jackson_28.wav',
 '0_yweweler_19.wav',
 '7_yweweler_36.wav',
 '9_nicolas_33.wav',
 '1_nicolas_26.wav',
 '5_theo_17.wav',
 '8_jackson_46.wav',
 '5_theo_22.wav',
 '0_nicolas_22.wav',
 '4_nicolas_46.wav',
 '1_jackson_14.wav',
 '2_jackson_34.wav',
 '9_theo_20.wav',
 '3_jackson_42.wav',
 '2_theo_28.wav',
 '9_yweweler_28.wav',
 '1_nicolas_49.wav',
 '1_yweweler_11.wav',
 '3_yweweler_2.wav',
 '7_yweweler_2.wav',
 '8_nicolas_41.wav',
 '8_jackson_37.wav',
 '3_nicolas_44.wav',
 '5_jackson_24.wav',
 '9_nicolas_1.wav',
 '0_jackson_48.wav',
 '1_nicolas_43.wav',
 '9_theo_38.wav',
 '1_jackson_1.wav',
 '9_nicolas_40.wav',
 '9_yweweler_39.wav',
 '6_jackson_19.wav',
 '1_jackson_0.wav',
 '7_jackson_1.wav',
 '1_nicolas_46.wav',
 '6_jackson_4.wav',
 '4_jackson_4.wav',
 '7_theo_8.wav',
 '2_theo_20.wav',
 '4_jackson_32.wav',
 '5_nicolas_8.wav',
 '1_jackson_47.wav',
 '2_theo_32.wav'

In [5]:
lst = []
for file_name in file_names :
#     print(file_name)
    sample, sample_rate = librosa.load(dir_path+file_name, sr=8000)
    mfccs = librosa.feature.mfcc(y=sample, sr=sample_rate, n_mfcc=24)
    lst.append([mfccs, file_name])

In [54]:
temp = []
for i in range(2000) :
    temp.append(lst[i][0].shape[1])
min_len_of_input = min(temp)
# lst[4][0][:,:3]

array([[-9.32457810e+01, -6.93924768e+01, -9.39204007e+01],
       [ 7.99284159e+01,  8.50041611e+01,  7.97162749e+01],
       [-5.30314555e+01, -5.82415348e+01, -5.77351824e+01],
       [-5.87995773e+01, -6.09486562e+01, -5.50983012e+01],
       [-1.61333771e+00, -7.01876683e+00, -1.12741386e+00],
       [ 8.60936959e+00,  4.82321608e+00, -3.67564859e+00],
       [-8.09811028e+00, -1.36009760e+01, -1.46511054e+01],
       [-1.11782875e+01, -1.03173376e+01, -8.47792898e+00],
       [-5.10675775e+00, -6.39543796e+00, -2.29058278e+00],
       [-9.34392564e+00, -7.66524062e+00, -8.30679642e+00],
       [-1.47595668e+01, -1.35143370e+01, -8.85947521e+00],
       [-1.64756284e+01, -1.30541063e+01, -6.34036502e+00],
       [-1.42291261e+00,  2.65511421e-02,  4.61138313e+00],
       [ 1.90148150e+00,  3.86143394e+00,  3.36767513e+00],
       [-1.99593096e+01, -2.09961096e+01, -1.42940760e+01],
       [-5.42826628e+00, -6.75847876e+00, -4.37553761e+00],
       [-6.15912210e+00, -6.80931576e+00

In [56]:
def purify(inmfcc, maxlength) :
    """
    :inmfcc : input mfcc and file name
    :rtype: truncated mfcc
    """
    outmfcc = inmfcc[0][:,:3]
    return outmfcc

In [61]:
res = []
N_samples = 2000

for i in lst :
#     print(i)
    res.append([purify(i, min_len_of_input), i[1]])
    

In [62]:
res

[[array([[-2.75459075e+02, -2.42056490e+02, -2.93408804e+02],
         [ 1.02330025e+01,  1.42477066e+01,  2.20863607e+01],
         [ 6.52528173e+01,  6.68541309e+01,  6.95231086e+01],
         [ 1.05294799e+01,  1.16069198e+01,  1.28038085e+01],
         [-4.70389200e+01, -4.75872408e+01, -4.55654841e+01],
         [ 2.84708565e+00,  1.06387747e+00,  1.70664038e-01],
         [-3.74512147e+01, -4.05786529e+01, -4.45827950e+01],
         [-1.49197018e+01, -1.90031609e+01, -2.34128672e+01],
         [-1.57527756e+01, -1.98231106e+01, -2.48731021e+01],
         [ 1.89170186e+00, -3.14345530e+00, -9.31139923e+00],
         [-8.43735183e+00, -1.17369461e+01, -1.57675579e+01],
         [ 1.22152035e+01,  9.54547930e+00,  5.92671088e+00],
         [-1.64177898e+00, -2.62891410e+00, -5.19880386e+00],
         [ 5.81091105e+00,  7.40515757e+00,  9.49650757e+00],
         [-7.65900323e-01,  8.16654842e-01,  2.13491416e+00],
         [-4.45137246e+00, -4.18709271e+00, -3.33403907e+00],
        

In [64]:
X = []
Y = []
for i in res :
    X.append(i[0])
    Y.append(int(i[1][0]))
X = np.asarray(X)
tempY = np.asarray(Y)

In [65]:
lsttmp = []
for i in tempY :
    tmp = []
    for numbers in range(10) :
        if numbers == i :
            num = 1
        else :
            num = 0
        tmp.append(num)
    lsttmp.append(tmp)


In [66]:
Y = np.asarray(lsttmp)
Y

array([[0, 0, 0, ..., 0, 1, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 1, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0]])

In [98]:
X.resize(2000,24,3,1)
X.shape

(2000, 24, 3, 1)

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.33, random_state=42)
# X_train = X_train.reshape(1340, 8000, 1)

In [100]:
model = Sequential()
# model.add(Conv1D(filters=1, kernel_size=10, activation='relu', input_shape=(8000,)))
model.add(Conv2D(32,(2,2),     
                  input_shape=X_train.shape[1:],      
                  activation= 'relu',
                  padding='same')) 
# model.add(Dense(8000, activation='relu'))
#model.add(Conv2D(32, (2,2), padding='valid'))
model.add(MaxPooling2D(2,2))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))

In [101]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

In [102]:
model.fit(X_train,y_train, epochs=1000)

Epoch 1/1000
1340/1340 [==============================] - 2s 1ms/step - loss: 3.0939 - acc: 0.2216
Epoch 2/1000
1340/1340 [==============================] - 0s 106us/step - loss: 1.7798 - acc: 0.3843
Epoch 3/1000
1340/1340 [==============================] - 0s 102us/step - loss: 1.4410 - acc: 0.4724
Epoch 4/1000
1340/1340 [==============================] - 0s 100us/step - loss: 1.2165 - acc: 0.5649
Epoch 5/1000
1340/1340 [==============================] - 0s 104us/step - loss: 1.0135 - acc: 0.6299
Epoch 6/1000
1340/1340 [==============================] - 0s 103us/step - loss: 0.8601 - acc: 0.6888
Epoch 7/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.7685 - acc: 0.7239
Epoch 8/1000
1340/1340 [==============================] - 0s 102us/step - loss: 0.6630 - acc: 0.7649
Epoch 9/1000
1340/1340 [==============================] - 0s 106us/step - loss: 0.6108 - acc: 0.7836
Epoch 10/1000
1340/1340 [==============================] - 0s 108us/step - loss: 0.5319 - acc

1340/1340 [==============================] - 0s 110us/step - loss: 0.0503 - acc: 0.9828
Epoch 82/1000
1340/1340 [==============================] - 0s 101us/step - loss: 0.0351 - acc: 0.9873
Epoch 83/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0401 - acc: 0.9896
Epoch 84/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0510 - acc: 0.9821
Epoch 85/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0393 - acc: 0.9836
Epoch 86/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0369 - acc: 0.9873
Epoch 87/1000
1340/1340 [==============================] - 0s 96us/step - loss: 0.0612 - acc: 0.9806
Epoch 88/1000
1340/1340 [==============================] - 0s 102us/step - loss: 0.0233 - acc: 0.9910
Epoch 89/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0304 - acc: 0.9896
Epoch 90/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0346 - acc: 0.9881
E

1340/1340 [==============================] - 0s 98us/step - loss: 0.0195 - acc: 0.9940
Epoch 162/1000
1340/1340 [==============================] - 0s 96us/step - loss: 0.0140 - acc: 0.9963
Epoch 163/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0044 - acc: 0.9985
Epoch 164/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0245 - acc: 0.9910
Epoch 165/1000
1340/1340 [==============================] - 0s 103us/step - loss: 0.0309 - acc: 0.9925
Epoch 166/1000
1340/1340 [==============================] - 0s 102us/step - loss: 0.0147 - acc: 0.9933
Epoch 167/1000
1340/1340 [==============================] - 0s 101us/step - loss: 0.0293 - acc: 0.9948
Epoch 168/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0212 - acc: 0.9955
Epoch 169/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0136 - acc: 0.9948
Epoch 170/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0203 - acc

1340/1340 [==============================] - 0s 108us/step - loss: 0.0044 - acc: 0.9985
Epoch 241/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0183 - acc: 0.9940
Epoch 242/1000
1340/1340 [==============================] - 0s 103us/step - loss: 0.0091 - acc: 0.9978
Epoch 243/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0077 - acc: 0.9963
Epoch 244/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0173 - acc: 0.9963
Epoch 245/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0177 - acc: 0.9940
Epoch 246/1000
1340/1340 [==============================] - 0s 103us/step - loss: 0.0130 - acc: 0.9955
Epoch 247/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0112 - acc: 0.9970
Epoch 248/1000
1340/1340 [==============================] - 0s 104us/step - loss: 0.0032 - acc: 0.9993
Epoch 249/1000
1340/1340 [==============================] - 0s 109us/step - loss: 0.0069 -

1340/1340 [==============================] - 0s 106us/step - loss: 0.0173 - acc: 0.9978
Epoch 320/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0103 - acc: 0.9970
Epoch 321/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0230 - acc: 0.9948
Epoch 322/1000
1340/1340 [==============================] - 0s 94us/step - loss: 0.0060 - acc: 0.9970
Epoch 323/1000
1340/1340 [==============================] - 0s 96us/step - loss: 0.0044 - acc: 0.9978
Epoch 324/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0088 - acc: 0.9963
Epoch 325/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0065 - acc: 0.9963
Epoch 326/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0042 - acc: 0.9985
Epoch 327/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0134 - acc: 0.9963
Epoch 328/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0129 - acc

...

1340/1340 [==============================] - 0s 105us/step - loss: 0.0092 - acc: 0.9993
Epoch 637/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0027 - acc: 0.9985
Epoch 638/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0052 - acc: 0.9978
Epoch 639/1000
1340/1340 [==============================] - 0s 105us/step - loss: 0.0016 - acc: 0.9993
Epoch 640/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0049 - acc: 0.9978
Epoch 641/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0014 - acc: 0.9993
Epoch 642/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0051 - acc: 0.9978
Epoch 643/1000
1340/1340 [==============================] - 0s 101us/step - loss: 0.0024 - acc: 0.9978
Epoch 644/1000
1340/1340 [==============================] - 0s 105us/step - loss: 0.0037 - acc: 0.9985
Epoch 645/1000
1340/1340 [==============================] - 0s 107us/step - loss: 0.0125 - 

1340/1340 [==============================] - 0s 110us/step - loss: 0.0040 - acc: 0.9985
Epoch 715/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0025 - acc: 0.9993
Epoch 716/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0039 - acc: 0.9993
Epoch 717/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0067 - acc: 0.9985
Epoch 718/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0017 - acc: 0.9993
Epoch 719/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0051 - acc: 0.9985
Epoch 720/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0023 - acc: 0.9993
Epoch 721/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0050 - acc: 0.9970
Epoch 722/1000
1340/1340 [==============================] - 0s 102us/step - loss: 0.0096 - acc: 0.9963
Epoch 723/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0126 - acc: 

1340/1340 [==============================] - 0s 114us/step - loss: 0.0022 - acc: 0.9993
Epoch 794/1000
1340/1340 [==============================] - 0s 102us/step - loss: 0.0025 - acc: 0.9985
Epoch 795/1000
1340/1340 [==============================] - 0s 104us/step - loss: 2.8681e-04 - acc: 1.0000
Epoch 796/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0175 - acc: 0.9955
Epoch 797/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0020 - acc: 0.9985
Epoch 798/1000
1340/1340 [==============================] - 0s 104us/step - loss: 0.0029 - acc: 0.9993
Epoch 799/1000
1340/1340 [==============================] - 0s 104us/step - loss: 0.0120 - acc: 0.9955
Epoch 800/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0096 - acc: 0.9955
Epoch 801/1000
1340/1340 [==============================] - 0s 102us/step - loss: 0.0028 - acc: 0.9985
Epoch 802/1000
1340/1340 [==============================] - 0s 102us/step - loss: 3.

1340/1340 [==============================] - 0s 114us/step - loss: 0.0029 - acc: 0.9993
Epoch 873/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0066 - acc: 0.9985
Epoch 874/1000
1340/1340 [==============================] - 0s 103us/step - loss: 0.0241 - acc: 0.9948
Epoch 875/1000
1340/1340 [==============================] - 0s 101us/step - loss: 0.0051 - acc: 0.9985
Epoch 876/1000
1340/1340 [==============================] - 0s 100us/step - loss: 0.0019 - acc: 0.9993
Epoch 877/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0022 - acc: 0.9985
Epoch 878/1000
1340/1340 [==============================] - 0s 99us/step - loss: 8.5211e-04 - acc: 0.9993
Epoch 879/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0057 - acc: 0.9993
Epoch 880/1000
1340/1340 [==============================] - 0s 97us/step - loss: 0.0045 - acc: 0.9985
Epoch 881/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0048 

1340/1340 [==============================] - 0s 102us/step - loss: 0.0196 - acc: 0.9955
Epoch 952/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0037 - acc: 0.9985
Epoch 953/1000
1340/1340 [==============================] - 0s 97us/step - loss: 6.9380e-04 - acc: 0.9993
Epoch 954/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0028 - acc: 0.9985
Epoch 955/1000
1340/1340 [==============================] - 0s 98us/step - loss: 0.0042 - acc: 0.9985
Epoch 956/1000
1340/1340 [==============================] - 0s 101us/step - loss: 0.0012 - acc: 0.9993
Epoch 957/1000
1340/1340 [==============================] - 0s 94us/step - loss: 0.0019 - acc: 0.9985
Epoch 958/1000
1340/1340 [==============================] - 0s 99us/step - loss: 0.0019 - acc: 0.9993
Epoch 959/1000
1340/1340 [==============================] - 0s 98us/step - loss: 9.8083e-04 - acc: 1.0000
Epoch 960/1000
1340/1340 [==============================] - 0s 96us/step - loss: 1.2006

In [103]:
y_pred = model.predict(X_test)
model.evaluate(X_test, y_test)

660/660 [==============================] - 0s 113us/step


[0.10576116222756272, 0.9833333333333333]

In [116]:
jktest = res[1024][0]
res[1024][1]

'4_yweweler_9.wav'

In [110]:
jk_pred = model.predict(jktest.reshape(1,24,3,1))

In [113]:
jk_pred

array([[3.3309954e-35, 9.1025772e-14, 6.1200556e-26, 5.6658505e-26,
        1.0000000e+00, 1.3835547e-14, 5.3106179e-15, 2.9264593e-19,
        7.0119940e-19, 6.1062663e-21]], dtype=float32)

In [121]:
Y[1024]

array([0, 0, 0, 0, 1, 0, 0, 0, 0, 0])